In [1]:
# 规范化变量统计

In [1]:
# 导入程序包
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()

# 变量统计

In [36]:
df_model=pd.read_excel(project_path+'/data/processed_data/df_1.10.6_人工删除不重要列.xlsx')
if 'Unnamed: 0' in df_model.columns:
    df_model = df_model.drop(['Unnamed: 0'], axis=1)

In [37]:
df_model['日剂量']=df_model.日剂量.apply(lambda x: 1 if x==60 else 0)

In [38]:
df_model.shape

(211, 51)

In [39]:
df_model.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDSTXT',
       '日剂量', '冠心病', '肺部感染', '心功能不全', '脑梗死', '房颤', '心肌梗塞', '呼吸系统疾病_label',
       '神经系统疾病_label', '泌尿系统疾病_label', '内分泌和代谢疾病_label', '住院日期', '完成日期',
       '住院时长', '年龄', '性别', '基础疾病', '降脂药', '降血小板药', '红细胞_入院', '血红蛋白_入院',
       '白细胞_入院', '中性粒细胞计数_入院', '血小板_入院', '红细胞压积_入院', 'ALT_入院', 'AST_入院',
       'GGT_入院', '低密度脂蛋白（LDL）_入院', '前白蛋白_入院', '尿素氮（BUN）_入院', '总胆红素（TBil）_入院',
       '甘油三酯_入院', '白蛋白_入院', '直接胆红素（DBil）_入院', '肌酐（Cr）_入院', '胆固醇_入院', '血氯_入院',
       '血钠_入院', '血钾_入院', '高密度脂蛋白胆固醇（HDLC）_入院', 'PT_入院', 'INR_入院', 'APTT_入院'],
      dtype='object')

In [40]:
# 删除无效变量
df_model=df_model.drop(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间','EXDSTXT','心功能不全', 
                        '呼吸系统疾病_label',
       '神经系统疾病_label', '泌尿系统疾病_label', '内分泌和代谢疾病_label','住院日期', '完成日期',],axis=1)

In [41]:
df_model.shape

(211, 38)

In [42]:
# 选取变量
discrete_col = ['日剂量','冠心病', '肺部感染', '脑梗死', '房颤', '心肌梗塞', '性别',
       '基础疾病', '降脂药', '降血小板药', ]
continuous_col=[x for x in df_model.columns if x not in discrete_col]

## 分类变量统计

In [43]:
# 求分类变量比例
df_discrete_stat=pd.DataFrame(columns=['变量名称','所有事件(%d)' % df_model.shape[0],'缺失率(%)'])
for i in discrete_col:
    print(i)
    # 缺失率
    if df_model[i].isnull().sum()==0:
        miss_rate='0%'
    else:
        miss_rate=df_model[i].isnull().sum()/df_model.shape[0]
        miss_rate="%.2f%%" % (miss_rate * 100)      # 百分数输出
    df_discrete_stat.loc[df_discrete_stat.shape[0],['变量名称','缺失率(%)']]=[i+'，n(%)',miss_rate]
    
    # 分类变量单独统计
    name_list=[]
    num_perc_list=[]
    df_model_stat=df_model[df_model[i].notnull()].sort_values([i],ascending=True)
    
    # 二分类还是多分类变量
    if df_model_stat[i].nunique() <=2:
        if re.match('gender|性别',i):
            name_list=['男','女']
        else:
            name_list=['是','否']
        for name,value in zip(name_list,[1,0]):
            print(name)
            num=df_model_stat[df_model_stat[i]==value].shape[0]
            percent=num/df_model.shape[0]
            percent="%.2f%%" % (percent * 100)
            num_percent=str(num)+'('+percent+')'
            num_perc_list.append(num_percent)
    else:
        for value in sorted(df_model_stat[i].unique()):
            print(value)
            name_list.append(value)
            num=df_model_stat[df_model_stat[i]==value].shape[0]
            percent=num/df_model.shape[0]
            percent="%.2f%%" % (percent * 100)
            num_percent=str(num)+'('+percent+')'
            num_perc_list.append(num_percent)
        
        
    df_temp = pd.DataFrame(data={'变量名称':name_list,
                                 '所有事件(%d)' % df_model.shape[0]:num_perc_list})

    df_discrete_stat=pd.concat([df_discrete_stat,df_temp],axis=0)
    df_discrete_stat=df_discrete_stat.reset_index(drop=True)

日剂量
是
否
冠心病
是
否
肺部感染
是
否
脑梗死
是
否
房颤
是
否
心肌梗塞
是
否
性别
男
女
基础疾病
是
否
降脂药
是
否
降血小板药
是
否


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [44]:
name_list

['是', '否']

In [46]:
# 保存分类变量统计
df_discrete_stat.to_excel(project_path+'/data/result/stat/df_2.1_分类变量统计_importance.xlsx')

## 连续变量统计

In [47]:
# 统计全变量体系各变量的平均数、上下四分位数、缺失率
feature_quarter_list=[]
mean_quarter_list=[]
feature_std_list=[]
mean_std_list=[]
miss_list=[]
for i in continuous_col:
    # 计算上下四分位、均值、标准差
    try:
        data = df_model[i].astype('float')
        stat_result = pd.DataFrame(data.describe())
        mean_value=stat_result.loc['mean',i]
        up_quarter=stat_result.loc['25%',i]
        down_quarter=stat_result.loc['75%',i]
        std_value=stat_result.loc['std',i]
    except:
        mean_value=np.nan
        up_quarter=np.nan
        down_quarter=np.nan
    # 计算缺失率
    if df_model[i].isnull().sum()==0:
        miss_rate='0%'
    else:
        miss_rate=df_model[i].isnull().sum()/df_model.shape[0]
        miss_rate="%.2f%%" % (miss_rate * 100)      # 百分数输出
    miss_list.append(miss_rate)
    # mean(quarter)
    feature_quarter_list.append(i+'，mean（IQR）')
    mean_quarter_list.append('%.2f(%.2f-%.2f)' % (mean_value,up_quarter,down_quarter))
    # mean(std)
    feature_std_list.append(i+'，mean±std')
    mean_std_list.append('%.2f±%.2f' % (mean_value,std_value))

In [48]:
df_continuous_quarter=pd.DataFrame(data={'特征':feature_quarter_list,
                        'mean_quarter_list':mean_quarter_list,
                        'miss_list':miss_list})
df_continuous_std=pd.DataFrame(data={'特征':feature_std_list,
                        'mean_std_list':mean_std_list,
                        'miss_list':miss_list})

In [49]:
df_continous_stat=pd.concat([df_continuous_quarter,df_continuous_std],axis=1)
# 保存连续变量统计
df_continous_stat.to_excel(project_path+'/data/result/stat/df_2.2_连续变量统计_importance.xlsx')